# Delete duplicate values from POI

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/Repos/observatorio-ciudades/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Read data

In [41]:
data_dir = '../../../data/raw/santiago_data/'

rename_dict = {'BANCOS':'banco', 'Carnicerias':'carniceria',
              'Farmacias5':'farmacia', 'Ferreteria':'hogar',
                'libreria':'librerias', 'Minimarkets_y_Almacenes_pequenos':'local_mini_market',
              'PANADERIA':'bakeries', 'Restaurant':'restaurantes_bar_cafe'}

treat_dict = {('restaurantes_bar_cafe','hogar', 'local_mini_market', 'banco','supermercado','club_deportivo','farmacia',
              'librerias','bakeries'):['rut','empresa','geometry'],
           ('carniceria',
           'cines',
           'jardin_inf_priv',):['rut','rzn_scl','geometry'],
             ('eq_deportivo_priv','eq_deportivo_pub'):['nombre','sector','geometry']}

grl_dir = '../../../data/processed/00_pois_formated/'

In [17]:
aoi = aup.gdf_from_db('santiago_aoi','project_research')

OperationalError: (psycopg2.OperationalError) could not translate host name "None" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [45]:
for data_name in rename_dict.keys():
    aup.log(f'Reading data for {data_name}')
    gdf = gpd.read_file(data_dir + data_name + '.shp')

    gdf = gdf.to_crs("EPSG:4326")
    
    # gdf_clip = gpd.clip(gdf, aoi)
    gdf_clip = gdf.copy()

    for key in treat_dict.keys():
        # aup.log(key)
        if rename_dict[data_name] in key:
            print(rename_dict[data_name])
            print(gdf_clip.columns)
            gdf_clip.columns= gdf_clip.columns.str.lower()
            aup.log(f'Current data for {rename_dict[data_name]} is {len(gdf_clip)}')
            gdf_clip = gdf_clip.drop_duplicates(treat_dict[key])
            aup.log(f'After processing data for {rename_dict[data_name]} is reduced to {len(gdf_clip)}')
            gdf_clip.to_file(grl_dir+rename_dict[data_name]+'.gpkg')

banco
Index(['RUT', 'DV', 'VIGENCI', 'FECHA', 'TIPO_DI', 'CALLE', 'NUMERO', 'BLOQUE',
       'DEPARTA', 'VILLA_P', 'CIUDAD', 'COMUNA', 'REGION', 'sngldrs', 'lon_1',
       'lat_1', 'empresa', 'geometry'],
      dtype='object')
carniceria
Index(['RUT', 'DV', 'VIGENCI', 'FECHA', 'TIPO_DI', 'CALLE', 'NUMERO', 'BLOQUE',
       'DEPARTA', 'VILLA_P', 'CIUDAD', 'COMUNA', 'REGION', 'Rzn_scl',
       'sngldrs', 'geometry'],
      dtype='object')
farmacia
Index(['RUT', 'DV', 'VIGENCI', 'FECHA', 'TIPO_DI', 'CALLE', 'NUMERO', 'BLOQUE',
       'DEPARTA', 'VILLA_P', 'CIUDAD', 'COMUNA', 'REGION', 'sngldrs',
       'empresa', 'geometry'],
      dtype='object')
hogar
Index(['RUT', 'DV', 'VIGENCI', 'FECHA', 'TIPO_DI', 'CALLE', 'NUMERO', 'BLOQUE',
       'DEPARTA', 'VILLA_P', 'CIUDAD', 'COMUNA', 'REGION', 'sngldrs',
       'empresa', 'geometry'],
      dtype='object')
librerias
Index(['RUT', 'DV', 'VIGENCI', 'FECHA', 'TIPO_DI', 'CALLE', 'NUMERO', 'BLOQUE',
       'DEPARTA', 'VILLA_P', 'CIUDAD', 'COMUNA',

In [47]:
source_list = []
for source in rename_dict.values():
    source_list.append(source)

In [48]:
source_list

['banco',
 'carniceria',
 'farmacia',
 'hogar',
 'librerias',
 'local_mini_market',
 'bakeries',
 'restaurantes_bar_cafe']

## Define required treatment

In [11]:
source_list = ['','','',
                   'supermercado','clinica_priv','clinica_pub',
                   'hospital_priv','hospital_pub','',
                   'consult_ado_priv','consult_ado_pub',
                   '','eq_deportivo_pub','',
                   'tax_collection','feria','ep_plaza_big',
                   '','museos_priv','museos_pub','sitios_historicos',
                   '','','','edu_basica_priv',
                   'edu_basica_pub','edu_media_priv','edu_media_pub',
                   '','jardin_inf_pub','edu_especial_priv',
                   'edu_especial_pub','bibliotecas']
treat_dict = {('supermercado','club_deportivo'):['rut','empresa','geometry'],
           ('carniceria','hogar','local_mini_market','farmacia',
           'banco','cines','restaurantes_bar_cafe','librerias',
           'jardin_inf_priv','bakeries'):['rut','empresa','geometry'],
             ('eq_deportivo_priv','eq_deportivo_pub'):['nombre','sector','geometry']}

## Data processing

In [13]:
grl_dir = '../../../data/processed/00_pois_formated/'

In [19]:
for key in treat_dict.keys():
    for source in key:
        gdf = gpd.read_file(grl_dir+source+'.gpkg')
        gdf.columns= gdf.columns.str.lower()
        aup.log(f'Current data for {source} is {len(gdf)}')
        gdf = gdf.drop_duplicates(treat_dict[key])
        aup.log(f'After processing data for {source} is reduced to {len(gdf)}')
        gdf.to_file(grl_dir+source+'.gpkg')